In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [3]:
sc_pro_2E=pd.read_csv('20241104-2cell-early-all-preprocessed.csv',index_col='Protein.Group')

In [5]:
pro_embryo_2E={}
for col in sc_pro_2E.columns:
    label=col.split('_')[1]
    if label in pro_embryo_2E:
        pro_embryo_2E[label].append(col)
    else:
        pro_embryo_2E[label]=[col]

In [7]:
from numpy import mean,median,std

In [9]:
sc_pro_2E_meaneach=sc_pro_2E.copy()
sc_pro_2E_meaneach=sc_pro_2E_meaneach.drop(sc_pro_2E.columns,axis=1)
for embryo in pro_embryo_2E:
    sc_pro_2E_meaneach['mean_2E_'+embryo] =mean(sc_pro_2E[pro_embryo_2E[embryo]],axis=1)

In [11]:
sc_pro_2E_nobev=sc_pro_2E.copy()

In [13]:
sc_pro_2E_nobev=sc_pro_2E.copy()
for i in range(sc_pro_2E_nobev.shape[0]):
    for j in range(sc_pro_2E_nobev.shape[1]):
        sc_pro_2E_nobev.iloc[i,j]=sc_pro_2E.iloc[i,j]-sc_pro_2E_meaneach.loc[sc_pro_2E_nobev.index[i],'mean_2E_'+sc_pro_2E_nobev.columns[j].split('_')[1]]

In [14]:
DAP_2E=pd.read_csv('DAP_2E.txt',index_col='gene')
pro_2E_cvtop500=[]
for i in range(DAP_2E.shape[0]):
    pro_2E_cvtop500.append(DAP_2E.index[i])

In [17]:
sc_pro_2E_nobev_cvtop500=sc_pro_2E_nobev.loc[pro_2E_cvtop500]
matrix_2E_nobev_cvtop500=sc_pro_2E_nobev_cvtop500.transpose().corr()

In [19]:
def correlation_cluster(corr, abs_corr=False):
    corr_dist = scidist.squareform(corr.values, checks=False)
    corr_dist = 1 - np.abs(corr_dist) if abs_corr else 1 - corr_dist
    gene_clusters = hcluster.linkage(corr_dist, method="average")
    gene_leaves = hcluster.leaves_list(gene_clusters)

    clustered_corr = corr.values[gene_leaves, :][:, gene_leaves]
    clustered_genes = [corr.columns[i] for i in gene_leaves]
    rv = pd.DataFrame(clustered_corr, 
                      columns=clustered_genes, index=clustered_genes)

    # transform the linkage matrix for the reordered columns
    new_indices = np.argsort(gene_leaves).astype(int)
    new_linkage = gene_clusters.copy()
    old0 = np.nonzero(new_linkage[:, 0] < len(gene_leaves))[0]
    old1 = np.nonzero(new_linkage[:, 1] < len(gene_leaves))[0]
    new_linkage[old0, 0] = new_indices[new_linkage[old0, 0].astype(int)]
    new_linkage[old1, 1] = new_indices[new_linkage[old1, 1].astype(int)]
    rv.linkage = new_linkage

    return rv

In [21]:
import scipy.cluster.hierarchy as hcluster
import scipy.spatial.distance as scidist

In [23]:
reshaped_matrix_2E_nobev_cvtop500=correlation_cluster(matrix_2E_nobev_cvtop500, abs_corr=False)

In [25]:
def boundaries2(cor, percentile, threshold, frozen=None):
    if frozen == None:
        frozen = [False for i in range(cor.shape[1])]
    def find_clusters(tree):
        if tree.is_leaf():
            if frozen[tree.id]:
                return ([tree.id, tree.id + 1], True)
            else:
                return ([tree.id, tree.id + 1], False)

        (left_clusters, left_frozen) = find_clusters(tree.left)
        (right_clusters, right_frozen) = find_clusters(tree.right)

        if left_frozen or right_frozen:
            return (left_clusters[:-1] + right_clusters, True)
        else:
            joint_values = cor.values[left_clusters[0]:left_clusters[-1], 
                                       right_clusters[0]:right_clusters[-1]].flatten()
            if np.sum(np.abs(joint_values) >= threshold) >= percentile*int(len(joint_values)) :
                return ([left_clusters[0], right_clusters[-1]], False)
            else:
                return (left_clusters[:-1] + right_clusters, False)

    tree = hcluster.to_tree(cor.linkage)
    (clusters, is_frozen) = find_clusters(tree)

    return clusters

In [27]:
Uclusters_2E_cvtop500_05 = boundaries2(reshaped_matrix_2E_nobev_cvtop500, 0.8, 0.5)
Lbdry_2E_cvtop500_05 = zip(Uclusters_2E_cvtop500_05[:-1], Uclusters_2E_cvtop500_05[1:])
Lbdry_large_2E_cvtop500_05 = [(a, b) for (a, b) in Lbdry_2E_cvtop500_05 if b - a >= 3]
sum_2E_cvtop500_05=0
for i in range(len(Lbdry_large_2E_cvtop500_05)):
    sum_2E_cvtop500_05+=(Lbdry_large_2E_cvtop500_05[i][1]-Lbdry_large_2E_cvtop500_05[i][0])
len(Lbdry_large_2E_cvtop500_05),sum_2E_cvtop500_05

(57, 228)

In [29]:
CPM_2E_cvtop500_05={}
for i in range(len(Lbdry_large_2E_cvtop500_05)):
    for j in range(Lbdry_large_2E_cvtop500_05[i][0],Lbdry_large_2E_cvtop500_05[i][1]):
        if i in CPM_2E_cvtop500_05:
            if ';' not in reshaped_matrix_2E_nobev_cvtop500.index[j]:
                CPM_2E_cvtop500_05[i].append(reshaped_matrix_2E_nobev_cvtop500.index[j])
            else:
                CPM_2E_cvtop500_05[i].append(reshaped_matrix_2E_nobev_cvtop500.index[j].split(';')[0])
        else:
            if ';' not in reshaped_matrix_2E_nobev_cvtop500.index[j]:
                CPM_2E_cvtop500_05[i]=[reshaped_matrix_2E_nobev_cvtop500.index[j]]
            else:
                CPM_2E_cvtop500_05[i]=[reshaped_matrix_2E_nobev_cvtop500.index[j].split(';')[0]]

In [31]:
CPM_dataframe_2E_cvtop500_05=pd.DataFrame(dict([(k,pd.Series(v)) for k, v in CPM_2E_cvtop500_05.items()]))
CPM_dataframe_2E_cvtop500_05.to_csv('CPM_2E_cvtop500_05.txt',sep='\t',index=False)